In [1]:
using DataFrames
using CSV
using Random
using StatsBase

Random.seed!(2472001)

# df = DataFrame(CSV.File("iris.data", header=false))
# iris_names = ["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"]
# rename!(df, iris_names)

df= DataFrame(CSV.File("PlayTennis.csv"))


,Outlook,Temperature,Humidity,Wind,Play Tennis
,String15,String7,String7,String7,String3
1,Sunny,Hot,High,Weak,No
2,Sunny,Hot,High,Strong,No
3,Overcast,Hot,High,Weak,Yes
4,Rain,Mild,High,Weak,Yes
5,Rain,Cool,Normal,Weak,Yes
6,Rain,Cool,Normal,Strong,No
7,Overcast,Cool,Normal,Strong,Yes
8,Sunny,Mild,High,Weak,No
9,Sunny,Cool,Normal,Weak,Yes


In [2]:
function train_test_spliter(data, train_Percent = 2/3)
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, train_Percent *n))
    test_idx = view(idx, (floor(Int, train_Percent *n)+1):n)
    data[train_idx,:], data[test_idx,:]
end


train_test_spliter (generic function with 2 methods)

In [3]:
function cal_total_entropy(data)
    class_list = unique(data[:,end])
    n = size(data,1)
    total_entropy = 0
    for class in class_list
        p = count(==(class),data[:,end]) / n
        total_entropy = total_entropy + (-p*log2(p))
    end
    return total_entropy
end

cal_total_entropy (generic function with 1 method)

In [4]:
#TODO: reconfig for iris dataset
function cal_entropy(feature_data)
    #=
    feature_data has 2 col:
        1- col values that sastify certain condition
        2- result col (last col)
    =#
    n = size(feature_data,1)
    entropy = 0

    unq = unique(feature_data[:,end])
    for class in unq
        entropy_class = 0
        p = count(==(class), feature_data[:,end]) / n
        entropy_class = -p*log2(p)
        entropy += entropy_class
    end
    return entropy
end


cal_entropy (generic function with 1 method)

In [5]:
function cal_infomation_gain(feature_name, data)
    #= 
    feature_name = name of feature
    data = whole dataframe
    =#
    feature_values = unique(data[:,feature_name])
    n = size(data,1)
    feature_info = 0.0

    for value in feature_values 
        feature_data = filter(n->n[feature_name] == value,data)
        feature_count = size(feature_data,1)
        feature_prob = feature_count / n
        feature_value_entropy = cal_entropy(feature_data)
        feature_info += feature_prob * feature_value_entropy
    end
    
    # return cal_entropy(data[:,[feature_name, names(data)[end]]]) - feature_info
    return cal_entropy(data) - feature_info
    
end


cal_infomation_gain (generic function with 1 method)

In [6]:
#TODO: modify for return feature 
function find_most_infomative_feature(data)

    feature_list = names(data)[1:end-1]

    max_inf_gain = -1
    max_feature = Nothing
    for feature in feature_list
        feature_info_gain = cal_infomation_gain(feature,data)
        if max_inf_gain < feature_info_gain
            max_inf_gain = feature_info_gain
            max_feature = feature
        end
    end

    return max_feature
end


find_most_infomative_feature (generic function with 1 method)

In [7]:
#TEST
function count_map(data, feature_name)
    unq = unique(data[:,feature_name])
    num_element = Vector{Int64}() 

    for value in unq
        push!(num_element, count(==(value),data[:,feature_name]))
    end

    return Dict((unq[i] => num_element[i] for i in range(1, length(unq))))
    #return DataFrame(Values = unq, count = num_element)
end


count_map (generic function with 1 method)

In [8]:
function generate_sub_tree(feature_name, data)
    feature_value_count_dic = countmap(data[:,feature_name])
    tree =Dict{Any,Any}()
    for (feature_value, count) in feature_value_count_dic
        feature_value_data = filter(n->n[feature_name] == feature_value, data)

        assigned_to_node = false

        for c in unique(data[:,end])
            class_count = size(filter(n->n[end] == c, feature_value_data),1)
            if class_count == count
                push!(tree, feature_value => c)
                data = filter(n->n[feature_name] != feature_value,data)
                assigned_to_node = true
            end
        end

        if assigned_to_node == false
            push!(tree, feature_value => "?")
        end

             
    
    end
    return tree, data
    
    end

generate_sub_tree (generic function with 1 method)

In [9]:
function make_tree(root, prev_feature_value, data)
    if size(data,1) != 0
        max_info_feature = find_most_infomative_feature(data)
        tree ,data = generate_sub_tree(max_info_feature,data)
        next_root = nothing

        if prev_feature_value != nothing
            root[prev_feature_value] = Dict{Any,Any}()
            #root[prev_feature_value][max_info_feature] = tree
            push!(root,prev_feature_value=>Dict{Any,Any}(max_info_feature => tree))
            next_root = root[prev_feature_value][max_info_feature]

        else
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        end

        for (node,branch) in next_root
            if branch == "?"
                feature_value_data = filter(n->n[max_info_feature] == node, data)
                make_tree(next_root, node, feature_value_data)
            end
        end
    end
end




make_tree (generic function with 1 method)

In [10]:
function id3(data)
    train_data = data
    tree = Dict{Any,Any}()
    make_tree(tree, nothing, train_data)

    return tree
    
end

id3 (generic function with 1 method)

In [11]:
function predict(tree, instance)
    if typeof(tree) != Dict{Any,Any}
        return tree
    else
        root_node = pop!(string.(keys(tree)))
        feature_value = instance[root_node]

        if feature_value in string.(keys(tree[root_node]))
            return predict(tree[root_node][feature_value], instance)
        else
            return nothing
        end
    end   
end

train, test =train_test_spliter(df)
tree =id3(train)
for i in eachrow(test)
    println(predict(tree,i))
    println("++")
    println(i[end])
    println("==================")
end

No
++
No
No
++
Yes
Yes
++
Yes
No
++
Yes
Yes
++
Yes


In [39]:
a = " > 0.3"
a = lstrip(a)
# a = split(a," ")
# print(a)

b = string(split(a)[1])
c = parse(Float64,string(split(a)[2]))
# println(c)
# println(typeof(c))
# println(b)
# println(typeof(b))
stringf = string(" ",b," ",string(c))

" > 0.3"